In [371]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [372]:
df = pd.read_csv('/kaggle/input/titanic/train.csv')

In [373]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [374]:
df['Age'].replace(to_replace = np.nan, value = np.mean(df['Age']),inplace=True)

In [375]:
df['Pclass'].replace(to_replace = [1,2,3], value = ['UpperClass','MiddleClass', 'LowerClass'],inplace=True)

In [376]:
pd.value_counts(df.Pclass)
Pclass = pd.get_dummies(df.Pclass)
df_new = pd.concat([df,Pclass], axis = 'columns')
df_new.drop(['Pclass'], axis='columns', inplace = True)
df_new.head()

,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,LowerClass,MiddleClass,UpperClass
0,1,0,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,0,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,0,1
2,3,1,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,0,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,0,1
4,5,0,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,0,0


In [377]:
df_new.isnull().sum()

PassengerId      0
Survived         0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
LowerClass       0
MiddleClass      0
UpperClass       0
dtype: int64

In [378]:
df1 = df_new[~df_new.Embarked.isnull()].copy()

In [379]:
df1.drop(['Ticket', 'Cabin', 'Name', 'PassengerId'], axis='columns', inplace = True)

In [380]:
df1['Sex'].replace(to_replace = ['male', 'female'], value = [1,0], inplace=True)

In [381]:
pd.value_counts(df1.Embarked)
Embarked = pd.get_dummies(df1.Embarked)
df2 = pd.concat([df1,Embarked], axis = 'columns')
df2.drop(['Embarked'], axis='columns', inplace = True)
df2.head()

,Survived,Sex,Age,SibSp,Parch,Fare,LowerClass,MiddleClass,UpperClass,C,Q,S
0,0,1,22.0,1,0,7.2500,1,0,0,0,0,1
1,1,0,38.0,1,0,71.2833,0,0,1,1,0,0
2,1,0,26.0,0,0,7.9250,1,0,0,0,0,1
3,1,0,35.0,1,0,53.1000,0,0,1,0,0,1
4,0,1,35.0,0,0,8.0500,1,0,0,0,0,1


In [382]:
df2['Fare'] =  df2['Fare'] / np.max(df2['Fare'])

In [383]:
df2.isnull().sum()

Survived       0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
LowerClass     0
MiddleClass    0
UpperClass     0
C              0
Q              0
S              0
dtype: int64

In [384]:
df2.columns

Index(['Survived', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'LowerClass',
       'MiddleClass', 'UpperClass', 'C', 'Q', 'S'],
      dtype='object')

In [385]:
X = df2[['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'LowerClass',
       'MiddleClass', 'UpperClass', 'C', 'Q', 'S']]
X.shape

(889, 11)

In [386]:
y = np.array(df2[['Survived']]).reshape(-1)
y.shape

(889,)

In [387]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

In [388]:
from keras.models import Sequential
from keras.layers import Dense

In [351]:
# define the keras model
model = Sequential()
model.add(Dense(15, input_dim=11, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [352]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [353]:
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10, verbose=0)

In [354]:
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

28/28 [==============================] - 0s 980us/step - loss: 0.4055 - accuracy: 0.7998
Accuracy: 79.98


In [279]:
# make class predictions with the model
from sklearn.metrics import accuracy_score
predictions = model.predict_classes(X_test).reshape(-1)
accuracy = accuracy_score(y_test, predictions, normalize=True, sample_weight=None)
accuracy

ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1462 predict_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1445 run_step  **
        outputs = mo

In [ ]:
# from sklearn.linear_model import LogisticRegression
# lr = LogisticRegression(penalty="none", random_state=42, max_iter=10000, solver="saga")
# lr.fit(X_train, y_train)
# train_score = lr.score(X_train, y_train)
# test_score = lr.score(X_test, y_test)
# train_score, test_score

In [355]:
test = pd.read_csv('/kaggle/input/titanic/test.csv')
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [356]:
test.drop(['Cabin','Ticket', 'Name'], axis='columns', inplace = True)
test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...,...
413,1305,3,male,NaN,0,0,8.0500,S
414,1306,1,female,39.0,0,0,108.9000,C
415,1307,3,male,38.5,0,0,7.2500,S
416,1308,3,male,NaN,0,0,8.0500,S


In [357]:
test.isnull().sum()

PassengerId     0
Pclass          0
Sex             0
Age            86
SibSp           0
Parch           0
Fare            1
Embarked        0
dtype: int64

In [358]:
test['Age'].replace(to_replace = np.nan, value = np.mean(test['Age']),inplace=True)

In [359]:
test['Fare'].replace(to_replace = np.nan, value = np.mean(test['Fare']),inplace=True)

In [360]:
test1 = test.copy()
test1.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [361]:
test1['Sex'].replace(to_replace = ['male', 'female'], value = [1,0],inplace=True)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Embarked,female,male
0,892,3,34.5,0,0,7.8292,Q,0,1
1,893,3,47.0,1,0,7.0000,S,1,0
2,894,2,62.0,0,0,9.6875,Q,0,1
3,895,3,27.0,0,0,8.6625,S,0,1
4,896,3,22.0,1,1,12.2875,S,1,0


In [362]:
test1['Pclass'].replace(to_replace = [1,2,3], value = ['UpperClass','MiddleClass', 'LowerClass'],inplace=True)

In [363]:
pd.value_counts(test1.Pclass)
Pclass = pd.get_dummies(test1.Pclass)
test3 = pd.concat([test1,Pclass], axis = 'columns')
test3.drop(['Pclass'], axis='columns', inplace = True)
test3.head()

,PassengerId,Age,SibSp,Parch,Fare,Embarked,female,male,LowerClass,MiddleClass,UpperClass
0,892,34.5,0,0,7.8292,Q,0,1,1,0,0
1,893,47.0,1,0,7.0000,S,1,0,1,0,0
2,894,62.0,0,0,9.6875,Q,0,1,0,1,0
3,895,27.0,0,0,8.6625,S,0,1,1,0,0
4,896,22.0,1,1,12.2875,S,1,0,1,0,0


In [364]:
pd.value_counts(test3.Embarked)
Embarked = pd.get_dummies(test3.Embarked)
test2 = pd.concat([test3,Embarked], axis = 'columns')
test2.drop(['Embarked'], axis='columns', inplace = True)
test2.head()

,PassengerId,Age,SibSp,Parch,Fare,female,male,LowerClass,MiddleClass,UpperClass,C,Q,S
0,892,34.5,0,0,7.8292,0,1,1,0,0,0,1,0
1,893,47.0,1,0,7.0000,1,0,1,0,0,0,0,1
2,894,62.0,0,0,9.6875,0,1,0,1,0,0,1,0
3,895,27.0,0,0,8.6625,0,1,1,0,0,0,0,1
4,896,22.0,1,1,12.2875,1,0,1,0,0,0,0,1


In [365]:
test2['Fare'] =  test2['Fare'] / (np.max(test2['Fare']))

In [366]:
test2.columns

Index(['PassengerId', 'Age', 'SibSp', 'Parch', 'Fare', 'female', 'male',
       'LowerClass', 'MiddleClass', 'UpperClass', 'C', 'Q', 'S'],
      dtype='object')

In [367]:
# X_pred = test2[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
#        'Fare', 'C', 'Q', 'S']]
X_pred = test2[['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'LowerClass',
       'MiddleClass', 'UpperClass', 'C', 'Q', 'S']]
X_pred.shape

(418, 12)

In [368]:
predictions = model.predict_classes(X_pred).reshape(-1)
predictions.shape

(418,)

In [369]:
result = pd.DataFrame()
result['Survived'] =  predictions
result['PassengerId'] = test2['PassengerId']
result.set_index("PassengerId", inplace = True)
result

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1
...,...
1305,0
1306,1
1307,0


In [370]:
result.to_csv('/kaggle/working/Predicted_Value.csv')